In [1]:
import numpy as np
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.data import Dataset, AUTOTUNE
from tensorflow import keras
import re
from keras import losses

2024-10-02 21:24:23.818149: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-02 21:24:23.825804: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-02 21:24:23.835236: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-02 21:24:23.837912: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-02 21:24:23.844460: I tensorflow/core/platform/cpu_feature_guar

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


I0000 00:00:1727893464.868099   27834 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727893464.884610   27834 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727893464.884654   27834 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [3]:
raw = open('data2.txt', mode='r', encoding='utf-8').readlines()
data = []
for line in raw:
    if line != '\n' and 'Глава' not in line:
        data.append(' '.join(line.split()[1:]))
    
data = [line.replace('\n', ' ').replace('\xa0', ' ') for line in data]
text = ' '.join(data)

In [4]:
text[:100]

'завет ЗАВЕТ Моисея  В начале сотворил Бог небо и землю. Земля же была безвидна и пуста, и тьма над б'

In [5]:
def get_features_target(seq):
    features = seq[:-1]
    target = seq[1:]
    return features, target

BATCH_SIZE = 32

word_pattern = r'[^а-яА-ЯёЁ0-9 :,-]'
clear_text = re.sub(word_pattern, '', text)

alphabet = np.array(sorted(set(clear_text.split(' '))))

clear_text = np.array(clear_text.split(' '))
clear_text = clear_text[clear_text != '']

word_to_idx = {}
idx_to_word = {}

for idx, word in enumerate(alphabet):
    word_to_idx[word] = idx
    idx_to_word[idx] = word

text_idx = np.array([word_to_idx[word] for word in clear_text])
sequences = Dataset.from_tensor_slices(text_idx).batch(BATCH_SIZE, drop_remainder=True)
dataset = sequences.map(get_features_target)

data = dataset.batch(BATCH_SIZE, drop_remainder=True).repeat()
data = data.prefetch(AUTOTUNE)

I0000 00:00:1727893465.228090   27834 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727893465.228138   27834 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727893465.228151   27834 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727893465.302793   27834 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727893465.302838   27834 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-10-02

In [6]:
model = keras.Sequential([
    keras.layers.Embedding(len(alphabet), 64),
    keras.layers.LSTM(64, return_sequences=True, stateful=True),
    keras.layers.Dense(len(alphabet), activation='relu'),
])

model.compile(optimizer='adam', loss=losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
model.fit(data, epochs=20, verbose=1, steps_per_epoch= len(sequences) // BATCH_SIZE)

Epoch 1/20
  1/651 ━━━━━━━━━━━━━━━━━━━━ 13:59 1s/step - accuracy: 0.0000e+00 - loss: 11.0893

2024-10-02 21:24:26.822850: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


651/651 ━━━━━━━━━━━━━━━━━━━━ 34s 51ms/step - accuracy: 0.0716 - loss: 9.7599
Epoch 2/20
651/651 ━━━━━━━━━━━━━━━━━━━━ 32s 49ms/step - accuracy: 0.0743 - loss: 9.0822
Epoch 3/20
651/651 ━━━━━━━━━━━━━━━━━━━━ 33s 50ms/step - accuracy: 0.0767 - loss: 9.0136
Epoch 4/20
651/651 ━━━━━━━━━━━━━━━━━━━━ 33s 50ms/step - accuracy: 0.0802 - loss: 8.9643
Epoch 5/20
651/651 ━━━━━━━━━━━━━━━━━━━━ 33s 50ms/step - accuracy: 0.0851 - loss: 8.9079
Epoch 6/20
651/651 ━━━━━━━━━━━━━━━━━━━━ 32s 50ms/step - accuracy: 0.0910 - loss: 8.8425
Epoch 7/20
651/651 ━━━━━━━━━━━━━━━━━━━━ 33s 50ms/step - accuracy: 0.0973 - loss: 8.7916
Epoch 8/20
651/651 ━━━━━━━━━━━━━━━━━━━━ 30s 47ms/step - accuracy: 0.1025 - loss: 8.7514
Epoch 9/20
651/651 ━━━━━━━━━━━━━━━━━━━━ 30s 46ms/step - accuracy: 0.1061 - loss: 8.7189
Epoch 10/20
651/651 ━━━━━━━━━━━━━━━━━━━━ 29s 45ms/step - accuracy: 0.1094 - loss: 8.6907
Epoch 11/20
651/651 ━━━━━━━━━━━━━━━━━━━━ 33s 50ms/step - accuracy: 0.1135 - loss: 8.6629
Epoch 12/20
651/651 ━━━━━━━━━━━━━━━━━━━━ 

In [7]:
def predict_next(sample, model, tokenizer, vocabulary, n_next, rnd_power, batch_size):
    sample_token = [tokenizer[char] for char in sample.split()]
    predicted = sample_token

    sample_tensor = tf.expand_dims(sample_token, 0)
    sample_tensor = tf.repeat(sample_tensor, batch_size, axis=0)
    
    for _ in range(n_next):
        cur = model(sample_tensor)
        cur = cur[0].numpy() / rnd_power
        cur = tf.random.categorical(cur, num_samples=1)[-1, 0].numpy()
        predicted.append(cur)
        sample_tensor = predicted[-99:]
        sample_tensor = tf.expand_dims([cur], 0)
        sample_tensor = tf.repeat(sample_tensor, batch_size, axis=0)
    res = [vocabulary[i] for i in predicted]
    generated = ' '.join(res)
    return generated

In [10]:
print(predict_next(
    sample='бог',
    model=model,
    tokenizer=word_to_idx,
    vocabulary=idx_to_word,
    n_next=20,
    temperature=0.6,
    batch_size=BATCH_SIZE,
    word=True
))

бог его и хула из земли и преславного подчиниться останься в любезно, что длина их в Самом Каину а на земле


In [11]:
print(predict_next(
    sample='1',
    model=model,
    tokenizer=word_to_idx,
    vocabulary=idx_to_word,
    n_next=20,
    temperature=0.6,
    batch_size=BATCH_SIZE,
    word=True
))

1 нелёгкое литых знанием, а ты и я и кто на земле И когда в потеряв который на него и для
